# Week 2

## Hill Cipher

In [10]:
function hill_transform(m::Matrix{Int}, message::Vector{Int})
    n = size(m, 1) # Assuming m is a square matrix
    # Pad message with 23
    pad_length = n - length(message) % n
    if pad_length < n
        message = vcat(message, fill(23, pad_length))
    end
    
    # Perform the transformation
    result = Int[]
    for i in 1:n:length(message)
        chunk = message[i:i+n-1]
        transformed = m * chunk
        append!(result, transformed)
    end
    
    # Apply modulo 26 to the result
    result = mod.(result, 26)
    return result
end

hill_transform (generic function with 2 methods)

### Check key and the inverse

$
Encipher: E = \begin{pmatrix}
6 & 24 & 1 \\
13 & 16 & 10 \\
20 & 17 & 15
\end{pmatrix}
Decipher: D = E^{-1} = \begin{pmatrix}
8 & 5 & 10 \\
21 & 8 & 21 \\
21 & 12 & 8
\end{pmatrix}
$

In [11]:
using LinearAlgebra

function inverse_modulo_n(E, n)
    detE = round(Int, det(E))
    detE_inv_mod = powermod(detE, -1, n)
    adjE = adjugate(E)
    invE_mod = mod.(detE_inv_mod * adjE, n)
    return invE_mod
end

function adjugate(E)
    C = cofactor(E)
    return C'
end

function cofactor(E)
    C = zeros(Int, size(E))
    for i in 1:size(E, 1)
        for j in 1:size(E, 2)
            minorE = E[1:end .!= i, 1:end .!= j]
            C[i, j] = (-1)^(i+j) * round(Int, det(minorE))
        end
    end
    return C
end

cofactor (generic function with 1 method)

In [12]:
E = reshape([6, 24, 1, 13, 16, 10, 20, 17, 15], 3, 3)'
D = inverse_modulo_n(E, 26)

3×3 Matrix{Int64}:
  8   5  10
 21   8  21
 21  12   8

In [13]:
function convert_string_to_integers(plain_text::String)
    # Convert to uppercase and filter out non-[A..Z] characters
    filtered_text = filter(c -> 'A' <= c <= 'Z', uppercase(plain_text))
    # Convert characters to integers (0-25)
    int_vector = [Int(char - 'A') for char in filtered_text]
    return int_vector
end

function convert_integers_to_string(int_vector::Vector{Int})
    # Convert integers (0-25) to characters
    char_vector = [Char(i + 'A') for i in int_vector]
    # Collapse the array of characters into a string
    result_string = join(char_vector)
    return result_string
end

convert_integers_to_string (generic function with 1 method)

In [14]:
plain_text = "Hello, Meet me at the toga party!"
message = convert_string_to_integers(plain_text)

println(convert_string_to_integers(plain_text))
println(convert_integers_to_string(convert_string_to_integers(plain_text)))

[7, 4, 11, 11, 14, 12, 4, 4, 19, 12, 4, 0, 19, 19, 7, 4, 19, 14, 6, 0, 15, 0, 17, 19, 24]
HELLOMEETMEATTHETOGAPARTY


In [15]:
println(plain_text)
println(convert_string_to_integers(plain_text))
println(hill_transform(E, convert_string_to_integers(plain_text)))
cipher_text = convert_integers_to_string(hill_transform(E, convert_string_to_integers(plain_text)))
println(cipher_text)
println(hill_transform(D, convert_string_to_integers(cipher_text)))
println(convert_integers_to_string(hill_transform(D, convert_string_to_integers(cipher_text))))

Hello, Meet me at the toga party!
[7, 4, 11, 11, 14, 12, 4, 4, 19, 12, 4, 0, 19, 19, 7, 4, 19, 14, 6, 0, 15, 0, 17, 19, 24]
[19, 5, 9, 24, 19, 14, 9, 20, 17, 12, 12, 22, 5, 23, 2, 0, 2, 15, 25, 20, 7, 11, 20, 2, 17, 0, 20]
TFJYTOJURMMWFXCACPZUHLUCRAU
[7, 4, 11, 11, 14, 12, 4, 4, 19, 12, 4, 0, 19, 19, 7, 4, 19, 14, 6, 0, 15, 0, 17, 19, 24, 23, 23]
HELLOMEETMEATTHETOGAPARTYXX
